In [1]:
import pandas as pd

with open('data/WhatsApp Chat with Janice Quach.txt') as file:
    data = file.readlines()

d = {
    'date': [],
    'time': [],
    'person': [],
    'message': []
}
for i, row in enumerate(data[1:]):
    date_ors = row.split(',')
    date = date_ors[0].strip()
    time_ors = ''.join(date_ors[1:]).strip().split('-')
    time = time_ors[0].strip()
    person_ors = ''.join(time_ors[1:]).strip().split(':')
    person = person_ors[0].strip()
    message = ''.join(person_ors[1:]).strip()
    d['date'].append(date)
    d['time'].append(time)
    d['person'].append(person)
    d['message'].append(message)

df = pd.DataFrame(d)
df = df[df.person.str.contains('tyson') | df.person.str.contains('Janice')]
df = df[['person', 'message']]

messages_clean = []
messages = df.values
for m in messages:
    person = m[0]
    message = m[1]
    person = 'janice' if 'Janice' in person else person
    combined = f'{person}: {message}'
    if '<Media omitted>' in message:
        continue
    if 'https//' in message:
        continue
    messages_clean.append(combined)

for m in messages_clean:
    print(m)

janice: Clarentone
janice: What time is the dinner on Sunday o
janice: It’s here
janice: It came early
tyson: Won't make it
tyson: Was looking everywhere for canvas bag
janice: Accidentally put them all in your car forgetting to leave some
tyson: Sigh
tyson: May as well uber to work
tyson: 21 minute wait
janice: Walk to station
tyson: I'm ubering
janice: I’ll get there by 915 I think
janice: I’m near the bus stop
janice: On the side
janice: Before the light
tyson: Woe
tyson: Tesla is going nuts
tyson: $570 right now
tyson: Remember when Elon said he would take it private @ $420/share
tyson: That was when the price was about $300
janice: Omg 126%
tyson: It's fucking close to double now
tyson: Tesla has paid off so well
tyson: Jesus christ
tyson: If you wanted to cash out it wouldn't be a bad time
tyson: Or rebalance
tyson: Rebalance = sell some Tesla and reinvest in other stocks or assets
tyson: Long term investors typically rebalance every 6 months I think
janice: Ohhh
janice: Can we t

In [2]:
from transformers import pipeline, set_seed

set_seed(42)

prompt = input('janice: ')
context = '\n'.join(messages_clean[-20:])

text = f'''
{context}
janice: {prompt}
tyson:
'''

generator = pipeline('text-generation', model='gpt2', pad_token_id=50256)

sentences = generator(text, do_sample=True, top_k=50, temperature=0.22, max_new_tokens=50, num_return_sequences=3)
responses = []
for sentence in sentences:
    response = sentence['generated_text']
    response = response.replace(f'{context}\njanice: {prompt}\n', '')
    response = response.split('\n')[2]
    responses.append(response)
    
for i, r in enumerate(responses):
    print(r, f'[{i}]')

janice: hey tys
tyson: I'm not sure if you have any idea how long it took me to get here [0]
tyson: I'm not sure what you're talking about [1]
tyson: I'm not sure if I'm going to be able to read [2]


### Finetuning huggingface transformers
https://huggingface.co/transformers/v4.8.2/training.html

In [3]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

Found cached dataset imdb (/Users/teebz/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
type(raw_datasets)

datasets.dataset_dict.DatasetDict

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [5]:
inputs = tokenizer(messages_clean, padding="max_length", truncation=True)

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Loading cached processed dataset at /Users/teebz/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-f4a913bbd796bf7b.arrow
Loading cached processed dataset at /Users/teebz/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-7f0dc1f3009ea1fa.arrow
Loading cached processed dataset at /Users/teebz/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-20e8fc1863fccb42.arrow


In [7]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

Loading cached shuffled indices for dataset at /Users/teebz/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-5f9aefe4363c22c9.arrow
Loading cached shuffled indices for dataset at /Users/teebz/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-b3306cdcfb5b3071.arrow


In [14]:
type(full_eval_dataset)

datasets.arrow_dataset.Dataset

In [8]:
small_train_dataset.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset
)

In [12]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/teebz/opt/miniconda3/envs/python/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375
  Number of trainable parameters = 108311810


Step,Training Loss


KeyboardInterrupt: 